In [1]:
import qnmfits
import numpy as np
import scipy
import corner
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sys
from pathlib import Path

notebook_dir = Path().resolve()
sys.path.append(str(notebook_dir.parent))

notebook_dir = Path().resolve()
data_dir = notebook_dir.parent

from matplotlib.colors import to_hex
from matplotlib.lines import Line2D
from matplotlib.colors import LinearSegmentedColormap
from qnmfits.spatial_mapping_functions import * 
from bayes_qnm_GP_likelihood import *
from BGP_fits import BGP_fit
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [2]:
id = "0001"
sim_main = SXS_CCE(id, lev="Lev5", radius="R2")

n_max = 7

T0 = 0
T = 100

qnm_list = [(2,2,n,1) for n in np.arange(0, n_max+1)] #+ [(3,2,0,1)] + [(4,4,n,1) for n in np.arange(0, n_max+1)] + [(5,4,0,1)] 
spherical_modes = [(2, 2)] 

chif_mag_0 = sim_main.chif_mag
Mf_0 = sim_main.Mf

In [3]:
# Get the standard kernel parameters

with open(data_dir / 'bayes_qnm_GP_likelihood/data/tuned_params.pkl', 'rb') as f:
    params = pickle.load(f)

tuned_param_dict_main = params[id]

In [4]:
# The training / tuning is demonstrated in the `get_kernel_params` file and notebook. 

with open(data_dir / 'bayes_qnm_GP_likelihood/data/param_dict_sim_lm_full.pkl', 'rb') as f:
    param_dict_sim_lm = pickle.load(f)

tuning_hyperparams_s = [0.3239257812499994]  

hyperparam_rule_dict_s = {
    "sigma_max": "multiply",
}

tuned_param_dict_wn = {mode: get_new_params(param_dict_sim_lm[id][mode], tuning_hyperparams_s, hyperparam_rule_dict_s) for mode in param_dict_sim_lm[id]}

In [5]:
fit = BGP_fit(sim_main.times, 
            sim_main.h, 
            qnm_list, 
            Mf_0, 
            chif_mag_0, 
            T0, 
            tuned_param_dict_main, 
            kernel_main, 
            t0_method='closest', 
            T=T, 
            spherical_modes=spherical_modes,
            include_chif=True,
            include_Mf=True)

In [6]:
print(fit.covariance_matrix)

[[ 6.45000452e-06  3.85445630e-07 -5.72366193e-05 -6.93402260e-05
   7.94920194e-05  4.90076747e-04  2.41246555e-04 -1.47482954e-03
  -7.51367083e-04  2.48544319e-03  7.29689324e-04 -2.34966000e-03
  -3.07562469e-04  1.13862668e-03  5.95283181e-05 -2.21099035e-04
  -1.91447893e-06 -1.48908671e-06]
 [ 3.85445630e-07  9.59605975e-06  5.97965858e-05 -7.56250374e-05
  -4.38952367e-04  1.81038658e-04  1.34066092e-03 -1.02387890e-04
  -2.21511661e-03 -1.07786875e-04  1.99706332e-03  1.25458399e-04
  -9.17270872e-04 -4.25470347e-05  1.73888355e-04  1.24455763e-05
   2.96469841e-07 -2.86258391e-07]
 [-5.72366193e-05  5.97965858e-05  1.03864156e-03  1.31903934e-04
  -4.49610301e-03 -3.79090145e-03  8.99686201e-03  1.56308168e-02
  -1.13863530e-02 -2.94436060e-02  9.88110475e-03  2.83773105e-02
  -4.87773904e-03 -1.36701469e-02  9.03746863e-04  2.71196019e-03
   1.91217837e-05  1.16567510e-05]
 [-6.93402260e-05 -7.56250374e-05  1.31903934e-04  1.38914219e-03
   3.03368810e-03 -7.15527685e-03 -1.

In [9]:
BGP_fit.samples

AttributeError: type object 'BGP_fit' has no attribute 'samples'

In [7]:
fit_main = qnm_BGP_fit(
        sim_main.times,
        sim_main.h,
        qnm_list,
        Mf_0,
        chif_mag_0,
        T0,
        tuned_param_dict_main, 
        kernel_main, 
        t0_method="closest",
        T=T,
        spherical_modes=spherical_modes,
        include_chif=True,
        include_Mf=True,
    )

In [8]:
print(fit_main['covariance'])

[[ 6.45003764e-06  3.85464185e-07 -5.72366836e-05 -6.93407242e-05
   7.94908350e-05  4.90079253e-04  2.41251977e-04 -1.47483498e-03
  -7.51378313e-04  2.48544825e-03  7.29701863e-04 -2.34966315e-03
  -3.07569244e-04  1.13862864e-03  5.95296618e-05 -2.21099405e-04
  -1.91446233e-06 -1.48908794e-06]
 [ 3.85464185e-07  9.59605963e-06  5.97963322e-05 -7.56252468e-05
  -4.38951526e-04  1.81040597e-04  1.34066015e-03 -1.02394997e-04
  -2.21511656e-03 -1.07773080e-04  1.99706304e-03  1.25444485e-04
  -9.17270412e-04 -4.25401689e-05  1.73888275e-04  1.24441923e-05
   2.96460549e-07 -2.86263538e-07]
 [-5.72366836e-05  5.97963322e-05  1.03863860e-03  1.31906675e-04
  -4.49607805e-03 -3.79090398e-03  8.99678773e-03  1.56307879e-02
  -1.13862212e-02 -2.94434993e-02  9.88096112e-03  2.83771858e-02
  -4.87766017e-03 -1.36700886e-02  9.03731333e-04  2.71194811e-03
   1.91215331e-05  1.16567013e-05]
 [-6.93407242e-05 -7.56252468e-05  1.31906675e-04  1.38914927e-03
   3.03369365e-03 -7.15531995e-03 -1.